<h1 align="center"><font size="5">Applied Data Science Capstone Project</font></h1>

## Introduction

In this noted book we try to use machine learning calssification algorithms create model
for alanlyzing the data of Seattle coliision Severity.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

## About dataset

First we download the data set.

In [2]:
other_path = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"
df_raw = pd.read_csv(other_path)

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### We take a look of the data set frame

In [3]:
df_raw.head(10)  

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N
5,1,-122.387598,47.690575,6,320840,322340,E919477,Matched,Intersection,36974.0,...,Dry,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
6,1,-122.338485,47.618534,7,83300,83300,3282542,Matched,Intersection,29510.0,...,Wet,Daylight,NaN,8344002.0,NaN,10,Entering at angle,0,0,N
7,2,-122.320780,47.614076,9,330897,332397,EA30304,Matched,Intersection,29745.0,...,Dry,Daylight,NaN,NaN,NaN,5,Vehicle Strikes Pedalcyclist,6855,0,N
8,1,-122.335930,47.611904,10,63400,63400,2071243,Matched,Block,NaN,...,Dry,Daylight,NaN,6166014.0,NaN,32,One parked--one moving,0,0,N
9,2,-122.384700,47.528475,12,58600,58600,2072105,Matched,Intersection,34679.0,...,Dry,Daylight,NaN,6079001.0,NaN,10,Entering at angle,0,0,N


# Data visualization and pre-processing

In [4]:
df_raw.dtypes  
df_raw.shape  

(194673, 38)

### we take a detail look of the variable we want to analyze 

In [5]:
df_raw['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [6]:
df_raw['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [7]:
df_raw['ROADCOND'].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [8]:
df_raw['LIGHTCOND'].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [9]:
df_raw['SPEEDING'].value_counts()

Y    9333
Name: SPEEDING, dtype: int64

### Pre-processing:  Feature selection/extraction

After the pre look, we decide choose weather, road condition, light condition as the variable, severity as the taget to create the model

In [10]:
df_fresh = df_raw[['LIGHTCOND', 'ROADCOND', 'WEATHER', 'SEVERITYCODE']].dropna()
df_fresh.head()

,LIGHTCOND,ROADCOND,WEATHER,SEVERITYCODE
0,Daylight,Wet,Overcast,2
1,Dark - Street Lights On,Wet,Raining,1
2,Daylight,Dry,Overcast,1
3,Daylight,Dry,Clear,1
4,Daylight,Wet,Raining,2


In [11]:
print(df_fresh.SEVERITYCODE.value_counts())
print(df_fresh.shape)

1    132285
2     57052
Name: SEVERITYCODE, dtype: int64
(189337, 4)


### Balance the Dataset

We can see the target set is imbalance, so  we balance the data before modeling

In [12]:
from sklearn.utils import resample

In [13]:
df_1 = df_fresh[df_fresh.SEVERITYCODE == 1]
df_2 = df_fresh[df_fresh.SEVERITYCODE == 2]
df_1_dsample = resample(df_1,replace = False, n_samples = 57052,random_state = 4)
df_balance = pd.concat([df_1_dsample,df_2])
df_balance.SEVERITYCODE.value_counts()

2    57052
1    57052
Name: SEVERITYCODE, dtype: int64

### Convert Categorical features to numerical values

Also as sklearn package only can model numerical data, we use One Hot Encoding method to change the 
string type data to numeircal value

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
df_model_X = df_balance.drop(columns = 'SEVERITYCODE').dropna()

In [16]:
X_fit = df_model_X.values
X_fit

array([['Dusk', 'Wet', 'Overcast'],
       ['Daylight', 'Wet', 'Raining'],
       ['Dusk', 'Dry', 'Clear'],
       ...,
       ['Daylight', 'Dry', 'Clear'],
       ['Daylight', 'Dry', 'Clear'],
       ['Dusk', 'Dry', 'Clear']], dtype=object)

In [17]:
enc = OneHotEncoder()

### Transform and normalize the data

In [18]:
X = enc.fit_transform(X_fit).toarray()

print(X)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
y = df_balance['SEVERITYCODE'].values
y[0:5]

array([1, 1, 1, 1, 1])

# Decision Tree

For modeling, we apply decision tree first.

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (91283, 29) (91283,)
Test set: (22821, 29) (22821,)


In [22]:
CollisionTree = DecisionTreeClassifier(criterion="entropy", max_depth = 10)
CollisionTree # it shows the default parameters

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
CollisionTree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [24]:
predTree = CollisionTree.predict(X_test)
print (predTree [0:10])
print (y_test [0:10])

[2 2 1 2 2 2 2 2 2 2]
[1 2 2 1 1 2 1 2 1 1]


In [25]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

DecisionTrees's Accuracy:  0.5619385653564699


### Check the accuracy of decision tree

In [26]:
#Decison Tree Test
yhat_tree = CollisionTree.predict(X_test)

from sklearn.metrics import f1_score
f1_score_tree = f1_score(y_test, yhat_tree, average='weighted') 
print(f1_score_tree)

from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score_tree = jaccard_similarity_score(y_test, yhat_tree)
print(jaccard_similarity_score_tree)

0.537266541918114
0.5619385653564699


# Logistic Regression

Second we apply the logistic regression model

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
yhat_log = LR.predict(X_test)
print(yhat_log[0:10])
print (y_test[0:10])

[2 2 1 2 2 2 2 2 2 2]
[1 2 2 1 1 2 1 2 1 1]


In [29]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

array([[0.46236513, 0.53763487],
       [0.46236513, 0.53763487],
       [0.50433438, 0.49566562],
       ...,
       [0.47620971, 0.52379029],
       [0.46236513, 0.53763487],
       [0.50433438, 0.49566562]])

### check the accuracy of logistic regression model

In [30]:
#logistic regression Test
from sklearn.metrics import f1_score
f1_score_LR = f1_score(y_test, yhat_log, average='weighted')
print(f1_score_LR)

from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score_LR = jaccard_similarity_score(y_test, yhat_log)
print(jaccard_similarity_score_LR)

from sklearn.metrics import log_loss
log_loss_LR = log_loss(y_test, yhat_prob)
print(log_loss_LR)

0.5314368139211159
0.5611059988606985
0.6654749127580274


# Report

In [31]:
data = {'Algorithm':['Decision Tree','LogisticRegression'],'Jaccard':[jaccard_similarity_score_tree,jaccard_similarity_score_LR],
        'F1-score':[f1_score_tree,f1_score_LR],
        'LogLoss':[None,log_loss_LR]}
df1 = pd.DataFrame(data)
df1

,Algorithm,Jaccard,F1-score,LogLoss
0,Decision Tree,0.561939,0.537267,NaN
1,LogisticRegression,0.561106,0.531437,0.665475


## Conclusion

We can see that weather, light condition, road condition have certain connection with the severity of the car collision
in seattle city. 